In [4]:
!pip install fastapi uvicorn nest_asyncio transformers==4.36.2 -q

import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse, JSONResponse
from transformers import pipeline
import uvicorn

# -----------------------------------------------------------
# LOAD MODEL (fast + accurate)
# -----------------------------------------------------------
style_model = pipeline("text2text-generation", model="Vamsi/T5_Paraphrase_Paws")


# -----------------------------------------------------------
# FASTAPI APP
# -----------------------------------------------------------
app = FastAPI()


@app.get("/", response_class=HTMLResponse)
async def home():
    html = """
    <html>
    <head>
        <title>Story Style Transformer</title>
        <style>
            body {
                background: #1e293b;
                font-family: 'Segoe UI', sans-serif;
                color: white;
                padding: 40px;
            }
            h1 {
                text-align: center;
                color: #38bdf8;
                font-size: 40px;
            }
            .container {
                width: 85%%;
                margin: auto;
                background: #0f172a;
                padding: 30px;
                border-radius: 20px;
                box-shadow: 0 0 20px rgba(0,0,0,0.5);
            }
            textarea {
                width: 100%%;
                height: 150px;
                padding: 15px;
                border-radius: 10px;
                border: none;
                font-size: 16px;
            }
            select {
                width: 250px;
                padding: 10px;
                border-radius: 8px;
                font-size: 16px;
                background: #334155;
                color: white;
                border: none;
            }
            button {
                margin-top: 20px;
                padding: 12px 25px;
                background: #10b981;
                color: white;
                border: none;
                border-radius: 8px;
                font-size: 18px;
                cursor: pointer;
            }
            button:hover {
                background: #0d9d6f;
            }
            .output-box {
                margin-top: 30px;
                padding: 20px;
                background: #1e293b;
                border-radius: 10px;
                min-height: 120px;
                font-size: 17px;
            }
        </style>
    </head>

    <body>
        <h1>✨ Story Style Transformer</h1>

        <div class="container">
            <h2>Enter your story</h2>
            <textarea id="story"></textarea>

            <h2 style="margin-top:25px;">Choose a writing style</h2>
            <select id="style">
                <option value="sarcastic">Sarcastic</option>
                <option value="dramatic">Dramatic</option>
                <option value="professional">Professional</option>
                <option value="friendly">Friendly</option>
                <option value="motivational">Motivational</option>
            </select>

            <button onclick="transformStory()">Transform</button>

            <h2 style="margin-top:30px;">Transformed Story</h2>
            <div id="output" class="output-box">Your transformed text will appear here...</div>
        </div>

        <script>
            async function transformStory() {
                const story = document.getElementById("story").value;
                const style = document.getElementById("style").value;

                document.getElementById("output").innerHTML = "⏳ Processing...";

                const response = await fetch("/transform", {
                    method: "POST",
                    headers: { "Content-Type": "application/json" },
                    body: JSON.stringify({ story: story, style: style })
                });

                const result = await response.json();
                document.getElementById("output").innerHTML = result.transformed;
            }
        </script>
    </body>
    </html>
    """
    return HTMLResponse(html)


# -----------------------------------------------------------
# STYLE TRANSFORMATION API
# -----------------------------------------------------------
STYLE_PROMPTS = {
    "sarcastic": "Rewrite this in a sarcastic, witty tone:",
    "dramatic": "Rewrite this in a dramatic, emotional tone:",
    "professional": "Rewrite this in a formal professional tone:",
    "friendly": "Rewrite this in a warm friendly tone:",
    "motivational": "Rewrite this in a motivating, uplifting tone:"
}

@app.post("/transform")
async def transform(request: Request):
    data = await request.json()
    story = data.get("story")
    style = data.get("style")

    if not story:
        return JSONResponse({"transformed": "❌ No story provided."})

    prompt = STYLE_PROMPTS[style] + " " + story

    try:
        result = style_model(prompt, max_length=200)[0]["generated_text"]
    except Exception as e:
        result = "⚠ Transformation Error."

    return JSONResponse({"transformed": result})


# -----------------------------------------------------------
# START UVICORN SERVER (COLAB SAFE VERSION)
# -----------------------------------------------------------
config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)

print("🚀 Server is running! Click the link that appears below to open the web app.")
await server.serve()


2025-12-05 17:32:33.702 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 17:32:33.703 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 17:32:33.704 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 17:32:33.705 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 17:32:33.707 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 17:32:33.708 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 17:32:33.709 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 17:32:33.711 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar